# How to convert Axona data to NWB

In this notebook we show how to convert Axona raw (`.bin` + `.set`) or unit data (`.X`, `.pos`, `.eeg`, `.egf`) to NWB format. Since the Hussaini lab already has tools for extracting unit data from raw data, we implemented conversions based on `.eeg` or `.egf` and `.pos` files for LFP and position data respectively, rather than extracting the information directly from the `.bin` file. 

For the `.pos` file data this will allow the option for generating a `.pos` file from another recording setup that can then be added to an `nwb` file.

For convenience we made a few `NWBConverters` with various input files. Later we will also show how to add data from from e.g. a `.pos` file to an existing NWB file using the appropriate converter.

### Setting up the environment

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

In [ ]:
# For convenience, make sure to select a set-filename for which we also have raw, tetrode, eeg, egf and pos data.

filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.set'
filename_eeg = filename.replace('.set', '.eeg')
filename_egf = filename.replace('.set', '.egf')

base_dir = Path(filename).parent

In [ ]:
from hussaini_lab_to_nwb import (
    HussainiNWBConverter,
    HussainiUnitNWBConverter
)

1. __HussainiNWBConverter__: `.bin` + `.pos` + `.eeg` + `.set`

In [ ]:
# Set nwbfile name
nwbfile_name = 'nwb_bin_pos_lfp.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

In [ ]:
# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

2. __HussainiNWBConverter__: `.bin` + `.set`

In [ ]:
# Set nwbfile name
nwbfile_name = 'nwb_bin.nwb'


# Specify source data
source_data = dict(
    AxonaRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file
fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

3. __HussainiUnitNWBConverter__: `.X` + `.set`

In [ ]:
# Set nwbfile name
nwbfile_name = 'nwb_tetrode.nwb'


# Specify source data
source_data = dict(
    AxonaUnitRecordingExtractorInterface=dict(filename=filename)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

4. __HussainiNWBConverter__: `.pos` + `.set`

In [ ]:

# Set nwbfile name
nwbfile_name = 'nwb_pos.nwb'


# Specify source data
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

5. __HussainiNWBConverter__: `.eeg` + `.set`


In [ ]:

# Set nwbfile name
nwbfile_name = 'nwb_lfp_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

6. __HussainiNWBConverter__: `.egf` + `.set`


In [ ]:

# Set nwbfile name
nwbfile_name = 'nwb_lfp_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf)
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

7. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.eeg` + `.set`

In [ ]:

# Set nwbfile name
nwbfile_name = 'nwb_unit_eeg.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

8. __HussainiUnitNWBConverter__: `.X` + `.pos` + `.egf` + `.set`

In [ ]:

# Set nwbfile name
nwbfile_name = 'nwb_unit_egf.nwb'


# Specify source data
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_egf),
    AxonaUnitRecordingExtractorInterface=dict(filename=filename),
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiUnitNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

## Adding data to existing nwb files

Add `.pos` and `.eeg` data to `.bin` acquisition

__Note:__ We did not appropriately close the connection to the nwbfile earlier, and running the below code without doing so will result in an error. To avoid this, simply restart the notebook, evaluate the imports at the beginning and do not evaluate any of the above cells containing inspections of the nwbfiles with nwbwidgets.

In [ ]:
# Set nwbfile name
nwbfile_name = 'nwb_bin.nwb'


# Specify source data for pos
source_data = dict(
    AxonaPositionDataInterface=dict(filename=filename),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)



# Specify source data for eeg
source_data = dict(
    AxonaLFPDataInterface=dict(filename=filename_eeg),
)
print(json.dumps(source_data, indent=2))


# Initialize Converter
converter = HussainiNWBConverter(source_data=source_data)


# Get metadata
metadata = converter.get_metadata()


# Export to NWB file
output_file = base_dir / nwbfile_name

converter.run_conversion(
    metadata=metadata,
    nwbfile_path=output_file,
    overwrite=False,
    save_to_file=True,
    conversion_options=None
)


# Check NWB file

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    

%matplotlib inline

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)